In [1]:
import pymysql 
import sys

# 데이터베이스 접속 관련 정보 초기화
host ='localhost' # '127.0.0.1'
port = 3306
username = 'root'
password ='12345'
database = 'myproject'


In [2]:
# 접속상태 확인
conflag = True

print('데이터 베이스 연결 준비...')
try:
    conn = pymysql.connect(host=host,user=username,
                       passwd=password,db=database,
                       port=port, charset='utf8')
    print('데이터베이스 연결 성공')
except Exception as e:
    print('연결 실패', e)
    conflag=False
if conflag == False:
    sys.exit()


데이터 베이스 연결 준비...
데이터베이스 연결 성공


In [4]:
import pandas as pd
from tqdm import tqdm

# CSV 불러오기
df = pd.read_csv("폭염대피소_좌표.csv")
df.columns = [
    'name', 'address', 'latitude', 'longitude',
    'capacity', 'weekday_open', 'weekday_close',
    'weekend_open', 'weekend_close', 'member_only'
]

cursor = conn.cursor()  # 커서 생성
cnt = 0

for _, row in tqdm(df.iterrows(), total=len(df)):
    cnt += 1
    sql_ = f"""INSERT INTO heatwave_shelters 
    VALUES ({cnt}, 
        '{row['name']}', '{row['address']}', {row['latitude']}, {row['longitude']}, 
        {row['capacity'] if not pd.isna(row['capacity']) else 'NULL'},
        {'NULL' if pd.isna(row['weekday_open']) else "'" + row['weekday_open'] + "'"},
        {'NULL' if pd.isna(row['weekday_close']) else "'" + row['weekday_close'] + "'"},
        {'NULL' if pd.isna(row['weekend_open']) else "'" + str(row['weekend_open']) + "'"},
        {'NULL' if pd.isna(row['weekend_close']) else "'" + str(row['weekend_close']) + "'"},
        {int(row['member_only'])})
    """
    cursor.execute(sql_)

conn.commit()
print("CSV 데이터 삽입 완료")
cursor.close()
conn.close()


100%|██████████| 1002/1002 [00:00<00:00, 2557.72it/s]

CSV 데이터 삽입 완료


In [7]:
import pymysql

# 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='12345',
    db='myproject',
    port=3306,
    charset='utf8'
)

# 커서 생성
cursor = conn.cursor()

# SELECT 쿼리 실행
try:
    sql_ = "SELECT * FROM heatwave_shelters"
    cursor.execute(sql_)
    res = cursor.fetchall()
    
    for row in res:
        print(row)

except Exception as e:
    print("쿼리 실행 중 에러 발생:", e)

finally:
    cursor.close()
    conn.close()


(1, '동광동 원로의집', '부산광역시 중구 샘길 12', 35.1064381, 129.0329079, 25, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=61200), None, None, 1)
(2, '화목원로의집', '부산광역시 중구 대청북길 14 (대청동4가, 근영빌라)', 35.1051736, 129.0284258, 12, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=61200), None, None, 1)
(3, '죽화원로의집', '부산광역시 중구 망양로 252 (보수동1가, )', 35.1068055, 129.0258577, 8, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=61200), None, None, 1)
(4, '보수새마을원로의집', '부산광역시 중구 망양로319번길 26', 35.1073901, 129.0268245, 9, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=64800), None, None, 1)
(5, '보수대밭골 원로의집', '부산광역시 중구 고가길 32 (보수동1가)', 35.1036534, 129.0271168, 20, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=61200), None, None, 1)
(6, '부평동원로의집', '부산광역시 중구 중구로29번길 21 (부평동1가)', 35.1003497, 129.026764, 12, datetime.timedelta(seconds=46800), datetime.timedelta(seconds=61200), None, None, 1)
(7, '영주1동원로의집', '부산광역시 중구 초량상로7번길 11', 35.1132015, 129.034124

In [11]:
connection = pymysql.connect(
    host='localhost',
    port = 3306,
    user = 'root',
    password='12345',
    database='myproject',
    charset='utf8'
)

In [12]:
import pymysql
import requests
from datetime import datetime

# ✅ 인증키
decoded_key = "Y3CGt58/VVESD3/i/8/8UeArRvF3LXAnAiH0dERf863i9SMkBYv71v+tej8Vhv7mMh+q66hWQQJV18oZHe4LzA=="

# ✅ 현재 시간을 정각 또는 30분으로 내리기
now = datetime.now()
minute = 30 if now.minute >= 30 else 0
adjusted = now.replace(minute=minute, second=0, microsecond=0)

base_date = adjusted.strftime("%Y%m%d")
base_time = adjusted.strftime("%H%M")

# ✅ 격자 좌표 (부산 진구 기준)
nx = 35
ny = 129

# ✅ API 요청
url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
params = {
    "serviceKey": decoded_key,
    "pageNo": "1",
    "numOfRows": "1000",
    "dataType": "JSON",
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
}

response = requests.get(url, params=params)

try:
    items = response.json()["response"]["body"]["items"]["item"]
    tmp_items = [item for item in items if item["category"] == "TMP"]

    print(f"🌡️ TMP 항목 {len(tmp_items)}개 저장 시도 중...")

    # ✅ MySQL 연결 (⚠️ 여기만 본인 정보에 맞게 수정!)
    connection = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',         # ✔️ 'username' → 'user' 로 고침!
        password='12345',
        database='myproject',
        charset='utf8'
    )

    try:
        with connection.cursor() as cursor:
            for item in tmp_items:
                sql = """
                INSERT INTO weather_forecast (
                    base_date, base_time, fcst_date, fcst_time, category, value, nx, ny
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(sql, (
                    datetime.strptime(item['baseDate'], "%Y%m%d").date(),
                    datetime.strptime(item['baseTime'], "%H%M").time(),
                    datetime.strptime(item['fcstDate'], "%Y%m%d").date(),
                    datetime.strptime(item['fcstTime'], "%H%M").time(),
                    item['category'],
                    item['fcstValue'],
                    int(item['nx']),
                    int(item['ny'])
                ))

        connection.commit()
        print("✅ MySQL 저장 완료!")

    finally:
        connection.close()

except Exception as e:
    print("❌ 오류 발생:", e)
    print("📦 응답 원문:", response.text)


🌡️ TMP 항목 83개 저장 시도 중...
✅ MySQL 저장 완료!
